# Configuração de ambiente

Importação das bibliotecas:
- Pandas: para manipulação de dados através do alias "pd"
- Google Colab: para manipulação de arquivos no Google Drive

E execução do comando "%matplotlib inline" para que os gráficos sejam exibidos no notebook logo abaixo da célula de código que os gerou.

In [75]:
import pandas as pd
from google.colab import drive

%matplotlib inline

Carregamento do Drive para que o notebook possa acessar os arquivos do Google Drive e com o parâmetro "force_remount=True" para que o Drive seja montado a cada execução do notebook.

In [76]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Leitura e visualização do dataset

Leitura do arquivo "dataset.csv" através da função "read_csv" do Pandas e armazenamento do resultado na variável "df". Uso do parâmetro "sep" para informar qual o separador de colunas do arquivo e do parâmetro "encoding" para informar qual o tipo de codificação do arquivo.

In [77]:
csv_url = "https://github.com/viniciusgugelmin/data-science-3/blob/main/projects/valorant_stats/data/dataset.csv?raw=true"

df = pd.read_csv(csv_url, sep=',', encoding='utf-8', low_memory=False)

Atribuição do limite na exibição do dataset para 50 colunas e 20 linhas, afim de conseguir visualizar melhor os dados.

In [78]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

Exibição das 5 primeiras linhas do DataFrame através da função "head" do Pandas para verificar se o arquivo foi carregado corretamente e ter uma ideia do que ele contém.

In [80]:
df.head()

,region,name,tag,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,kills,deaths,assists,kd_ratio,kills_round,most_kills,score_round,wins,win_percent,agent_1,agent_2,agent_3,gun1_name,gun1_head,gun1_body,gun1_legs,gun1_kills,gun2_name,gun2_head,gun2_body,gun2_legs,gun2_kills,gun3_name,gun3_head,gun3_body,gun3_legs,gun3_kills
0,NaN,ShimmyXD,#NA1,Radiant,135.8,992,24.9,0,140,80,161,"1,506","1,408",703,1.07,0.7,29,208.8,59,59.6,Fade,Viper,Omen,Vandal,35,59,5,802,Phantom,33,62,5,220,Classic,36,60,3,147
1,NaN,XSET Cryo,#cells,Radiant,170.3,879,28.3,2,122,94,316,"1,608","1,187",206,1.35,1.0,32,270.6,52,65.8,Chamber,Jett,Raze,Vandal,41,56,3,689,Operator,8,91,0,226,Phantom,32,63,5,137
2,NaN,PuRelittleone,#yoruW,Radiant,147.5,720,24.0,3,117,59,216,"1,115","1,064",267,1.05,0.8,39,227.8,42,65.6,Yoru,Jett,Chamber,Vandal,38,57,4,444,Phantom,36,61,3,231,Operator,8,91,1,102
3,NaN,Boba,#0068,Radiant,178.2,856,37.3,3,83,49,235,"1,134",812,157,1.40,1.0,37,277.0,32,62.8,Jett,Chamber,KAY/O,Vandal,51,47,2,754,Sheriff,48,51,1,48,Phantom,44,56,0,36
4,NaN,i love mina,#kelly,Radiant,149.8,534,24.4,2,71,38,137,869,781,213,1.11,0.8,29,230.9,32,62.8,Jett,Raze,Chamber,Vandal,36,60,4,419,Spectre,21,71,8,65,Operator,8,92,0,64


# Processo inicial de limpeza

Apenas visualização das colunas

In [81]:
df.columns

Index(['region', 'name', 'tag', 'rating', 'damage_round', 'headshots',
       'headshot_percent', 'aces', 'clutches', 'flawless', 'first_bloods',
       'kills', 'deaths', 'assists', 'kd_ratio', 'kills_round', 'most_kills',
       'score_round', 'wins', 'win_percent', 'agent_1', 'agent_2', 'agent_3',
       'gun1_name', 'gun1_head', 'gun1_body', 'gun1_legs', 'gun1_kills',
       'gun2_name', 'gun2_head', 'gun2_body', 'gun2_legs', 'gun2_kills',
       'gun3_name', 'gun3_head', 'gun3_body', 'gun3_legs', 'gun3_kills'],
      dtype='object')

Antes de tudo resolvemos tirar jogadores que tenham menos de 10 vitórias, para assim evitarmos outliers, sabendo que para ser um jogador relevante ele acabará tendo muito mais que 10 vitórias, jogando ainda mais o jogo.

In [82]:
df = df[df['wins'] < 10]

Escolhemos tirar a região e etiqueta pois são dados não tão relevantes para esta pesquisa. Tiramos eliminações e vezes eliminadas pois já temos o percentual que é o cálculo que contém os dois números. Tiramos maior quantidade de eliminações e total de vitórias pois acaba sendo algo não tão relevante para o aprendizado, visto que os dados são bem parecidos na maioria dos casos e pode acabar mais confundindo do que ajudando. E por fim, tiramos dados de agentes e armas secundárias e terciárias pois não são tão relevantes.

In [83]:
df.drop(['region', 'tag', 'kills', 'deaths', 'most_kills', 'wins', 'agent_2', 'agent_3', 'gun1_kills'], inplace=True, axis=1)
df = df.drop(df.filter(regex='gun2_|gun3_|_head|_body|_legs').columns, axis=1)
df.head()

,name,rating,damage_round,headshots,headshot_percent,aces,clutches,flawless,first_bloods,assists,kd_ratio,kills_round,score_round,win_percent,agent_1,gun1_name
229,NYFU YaBoiDre,Immortal 3,159.8,222,28.9,1,24,12,58,70,1.05,0.8,236.0,52.9,Jett,Vandal
232,GUARD JonahP,Immortal 3,187.2,161,23.1,0,14,11,46,68,1.30,1.0,282.3,66.7,Raze,Vandal
314,Clanky,Immortal 3,128.7,126,26.7,0,14,13,21,53,1.20,0.7,199.6,81.8,Skye,Vandal
340,Neko,Immortal 3,146.8,223,31.8,0,32,13,46,79,1.07,0.8,223.1,52.9,Reyna,Vandal
354,dancingduck46253,Immortal 3,151.9,140,24.7,0,22,11,36,58,1.16,0.8,224.5,75.0,Chamber,Vandal


# Processo de limpeza de dados nulos e padronização de dados

Utilização do método **info** para tentar encontrar valores nulos que não deveriam ser nulos.

In [84]:
df_cleaned = df.copy()

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20060 entries, 229 to 85676
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              20058 non-null  object 
 1   rating            20060 non-null  object 
 2   damage_round      20060 non-null  float64
 3   headshots         20060 non-null  object 
 4   headshot_percent  20060 non-null  float64
 5   aces              20060 non-null  int64  
 6   clutches          20060 non-null  int64  
 7   flawless          20060 non-null  int64  
 8   first_bloods      20060 non-null  object 
 9   assists           20060 non-null  object 
 10  kd_ratio          20060 non-null  float64
 11  kills_round       20060 non-null  float64
 12  score_round       20060 non-null  float64
 13  win_percent       20060 non-null  float64
 14  agent_1           20060 non-null  object 
 15  gun1_name         20060 non-null  object 
dtypes: float64(6), int64(3), object(7)
mem

Acabamos percebendo que temos linhas com nomes de jogadores vazio, então resolvemos tirar mesmo que não seja algo relevante para o aprendizado supervisionado, acaba atrapalhando na comprovação e amostragem dos dados. Também retiramos colunas numéricas com valores igual a 0, que provavelmente são os nossos outliers.

In [85]:
df_cleaned = df_cleaned[df_cleaned['name'].notna()]
df_cleaned = df_cleaned[df_cleaned['headshots'] != "0"]
df_cleaned = df_cleaned[df_cleaned['headshot_percent'] != "0"]
df_cleaned = df_cleaned[df_cleaned['aces'] != "0"]
df_cleaned = df_cleaned[df_cleaned['clutches'] != "0"]
df_cleaned = df_cleaned[df_cleaned['flawless'] != "0"]
df_cleaned = df_cleaned[df_cleaned['first_bloods'] != "0"]
df_cleaned = df_cleaned[df_cleaned['assists'] != "0"]
df_cleaned = df_cleaned[df_cleaned['kd_ratio'] != "0"]
df_cleaned = df_cleaned[df_cleaned['kills_round'] != "0"]
df_cleaned = df_cleaned[df_cleaned['score_round'] != "0"]
df_cleaned = df_cleaned[df_cleaned['win_percent'] != "0"]
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19825 entries, 229 to 85676
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              19825 non-null  object 
 1   rating            19825 non-null  object 
 2   damage_round      19825 non-null  float64
 3   headshots         19825 non-null  object 
 4   headshot_percent  19825 non-null  float64
 5   aces              19825 non-null  int64  
 6   clutches          19825 non-null  int64  
 7   flawless          19825 non-null  int64  
 8   first_bloods      19825 non-null  object 
 9   assists           19825 non-null  object 
 10  kd_ratio          19825 non-null  float64
 11  kills_round       19825 non-null  float64
 12  score_round       19825 non-null  float64
 13  win_percent       19825 non-null  float64
 14  agent_1           19825 non-null  object 
 15  gun1_name         19825 non-null  object 
dtypes: float64(6), int64(3), object(7)
mem

Notamos também que existem algumas colunas que apresentam apenas números mas estão no formato errado, então fazemos a conversão.

In [86]:
df_cleaned['headshots'] = pd.to_numeric(df_cleaned['first_bloods'].str.replace(",","."), errors="coerce")
df_cleaned['first_bloods'] = pd.to_numeric(df_cleaned['first_bloods'].str.replace(",","."), errors="coerce")
df_cleaned['assists'] = pd.to_numeric(df_cleaned['assists'].str.replace(",","."), errors="coerce")
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19825 entries, 229 to 85676
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              19825 non-null  object 
 1   rating            19825 non-null  object 
 2   damage_round      19825 non-null  float64
 3   headshots         19825 non-null  int64  
 4   headshot_percent  19825 non-null  float64
 5   aces              19825 non-null  int64  
 6   clutches          19825 non-null  int64  
 7   flawless          19825 non-null  int64  
 8   first_bloods      19825 non-null  int64  
 9   assists           19825 non-null  int64  
 10  kd_ratio          19825 non-null  float64
 11  kills_round       19825 non-null  float64
 12  score_round       19825 non-null  float64
 13  win_percent       19825 non-null  float64
 14  agent_1           19825 non-null  object 
 15  gun1_name         19825 non-null  object 
dtypes: float64(6), int64(6), object(4)
mem

# Salvando dados limpos

Para salvar os dados limpos em um arquivo, vamos usar o método to_csv

In [87]:
df_cleaned.to_csv('/content/drive/MyDrive/UP/Ciência de Dados/Notebooks/data/dataset_clean.csv', index=False, sep=';')